In [6]:
from transformers import GPT2LMHeadModel
import os

os.environ['XDG_CACHE_HOME'] = './cache'
os.environ['HF_HOME'] = './cache'
os.environ['HF_DATASETS_CACHE'] = './cache'
os.environ['TRANSFORMERS_CACHE'] = './cache'
os.environ['HUGGINGFACE_HUB_CACHE'] = './cache'


In [7]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2", cache_dir='./cache')
sd_hf = model_hf.state_dict()

for k,v in sd_hf.items():
    print(k, v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [10]:
from transformers import pipeline, set_seed, PretrainedConfig

generator = pipeline('text-generation', model='gpt2', model_kwargs = {"cache_dir": './cache'})
set_seed(42)

Device set to use mps:0


In [12]:
generator("hello. I'm a model", max_length=30, num_return_sequences=10)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "hello. I'm a model, like he's a model, with these six things in his hand. Everything he's written with his imagination into the"},
 {'generated_text': 'hello. I\'m a model because no matter how many times you\'re thinking "OK," you\'re not going to get all of the answers back in'},
 {'generated_text': "hello. I'm a model with a very bright and bright future on my hands. This was a day for great things and I'm thankful for her"},
 {'generated_text': "hello. I'm a model. I'm human. I'm a human. You have my back. And you've got my mind. And I"},
 {'generated_text': 'hello. I\'m a model for the future."'},
 {'generated_text': "hello. I'm a model, a character, and I am a writer. There were some issues with the books, and I'll let you guess"},
 {'generated_text': "hello. I'm a model\n\nI'm one of the models that you can build\n\nThe first thing I'll do to know is that"},
 {'generated_text': 'hello. I\'m a model and I want to be a model because I love these things. I thin